In [35]:
import json, os, requests

### Query Params for eBay's "Completed Listings" API Endpoint

In [2]:
API_KEY = 'RobertBo-cinemaro-PRD-171ca9e35-af8bdcbb' # Enter your API Key/"App ID" Here. Mine was 40 chars long.

In [81]:
# FILM_CAMS = '15230'
ELEC_GUITARS = '33034' # Category code for electric guitars on eBay.
WATCHES = '14324'

In [4]:
USA = 'EBAY-US' # USA Marketplace only. API seems to start returning stuff from other markets eventually anyhow

In [5]:
USED = '3000' # Just the code for item condition "used". This is the focus of this project.

In [94]:
ITEM_FILTER_0 = f'itemFilter(0).name=Condition&itemFilter(0).value={USED}' # Only used guitars
ITEM_FILTER_1 = f'itemFilter(1).name=HideDuplicateItems&itemFilter(1).value=true' # No duplicate listings
ITEM_FILTER_2 = f'itemFilter(2).name=MinPrice&itemFilter(2).value=199' # Only items that sold for > this value
ITEM_FILTER_3 = f'itemFilter(3).name=MaxQuantity&itemFilter(3).value=1' # No lots or batch sales. One item at a time
ITEM_FILTER_4 = f'itemFilter(4).name=SoldItemsOnly&itemFilter(4).value=true' # Only looking at listings that sold.
ITEM_FILTER_5 = f'itemFilter(5).name=ListingType&itemFilter(5).value=Auction' # Only looking at auctions.
ITEM_FILTER_6 = f'itemFilter(6).name=MaxPrice&itemFilter(6).value=3500' # Only items that sold for < this value

In [95]:
FIND_COMPLETED = 'findCompletedItems' # This is the eBay API endpoint service we'll be querying.

### Actual Query Function

Here we go:

In [96]:
def find_completed(PAGE):
    '''Make a request to the eBay API and return the JSON text of this page number'''
    r = requests.get(f'https://svcs.ebay.com/services/search/FindingService/v1?'
                 f'OPERATION-NAME={FIND_COMPLETED}&'
                 f'X-EBAY-SOA-SECURITY-APPNAME={API_KEY}&'
                 f'RESPONSE-DATA-FORMAT=JSON&' # This value can be altered if you're not into JSON responses
                 f'REST-PAYLOAD&'
                 f'GLOBAL-ID={USA}&' # seems to prioritize the value you enter but returns other stuff too
                 f'categoryId={ELEC_GUITARS}&' # Product category goes here
                 f'descriptionSearch=true&' # More verbose responses
                 f'{ITEM_FILTER_0}&' # USED
                 f'{ITEM_FILTER_1}&' # NO DUPES
                 f'{ITEM_FILTER_2}&' # MINPRICE
                 f'{ITEM_FILTER_3}&' # NO LOTS
                 f'{ITEM_FILTER_4}&' # ONLY SOLD
                 f'{ITEM_FILTER_5}&' # AUCTIONS ONLY
                 f'{ITEM_FILTER_6}&' # MAX PRICE
                 f'paginationInput.pageNumber={str(PAGE)}&' # value to be looped through when collecting lotsa data
                 f'outputSelector=PictureURLLarge') # Why not grab the thumbnail URLs too. Could be cool
    return r.json()['findCompletedItemsResponse'][0]['searchResult'][0]['item']

### Which of these fields might be useful for a regression project?

* itemID, for passing to getSingleItem API


* NLP: Title


* CNN: Thumbnail images


* scalar: handlingTime
* scalar: Duration (derive from start/end cols)
* scalar: __convertedCurrentPrice__ - THIS IS THE INDEPENDENT VAR
* scalar: return time window


* ordinal: postal zip code, for fun kepler viz
* ordinal: Start day of week (derive from start col)
* ordinal: End day of week (derive from end col)


* 1-hot: Seller Country
* 1-hot: ShippingType
* 1-hot: expeditedShipping
* 1-hot: Returns Accepted

***

### What can we only get from the single-item API (below)?

* NLP: Description text
* NLP: Detailed __Condition__ Description
* NLP: SubTitle


* Scalar: Length of description text (non-html tags)
* Scalar: Number of images on eBay normal
* Scalar: Year of manufacture
* Scalar: Seller.FeedbackScore
* Scalar: Seller.PositiveFeedbackPercent
* Scalar: ReturnsWithin number of days
* Scalar: len(ExcludeShipToLocation) aka how many countries excluded?


* 1-hot (or NLP?): Brand
* 1-hot (or NLP?): Model
* 1-hot: Right/left handed
* 1-hot: Body color
* 1-hot: Body material
* 1-hot: String Config (6-string, etc)
* 1-hot: Body type (Solid / hollow, etc)
* 1-hot: Soundboard Style (arch / flat top etc)
* 1-hot: Country of manufacture
* 1-hot: Case
* 1-hot: NewBestOffer
* 1-hot: AutoPay

### Item-Specific Queries, for More Features

Beware, you only get 5000 queries a day, so don't loop on this too hard until you're persisting data.

In [98]:
def get_specs(ITEM_ID):
    '''Return the specifics of a single eBay auction. String input.'''
    r2 = requests.get('http://open.api.ebay.com/shopping?'
                    f'callname=GetSingleItem&'
                    f'responseencoding=JSON&'
                    f'appid={API_KEY}&'
                    f'siteid=0&' # USA Store
                    f'version=967&' # What is this?
                    f'ItemID={ITEM_ID}&' # Assigned above
                    f'IncludeSelector=Details,ItemSpecifics,TextDescription')
    try:
        return r2.json()['Item']
    except KeyError:
        pass

Seems like there's some variability between items when it comes to item specifics field.

### Persisting some Data for Analysis

Just write first page of listing results to .json files:

In [99]:
def persist_page_to_json(PAGE):
    '''Saves a page of JSON responses to one json per axe'''
    for i in range(len(PAGE)):
        with open("axe_listings/axe_%s.json" % (PAGE[i]['itemId'][0]), 'w') as f:  # writing JSON object
            json.dump(PAGE[i], f)

Now write one page of details to a JSON:

In [100]:
def persist_spec_to_json(spec):
    '''Writes one page of Axe Specs to one json'''
    try:
        with open("axe_specs/axe_%s.json" % (spec['ItemID']), 'w') as f:  # writing JSON object
            json.dump(spec, f)
    except TypeError:
        pass
    pass

Okay, careful, this is where we start to hammer the eBay API a little bit.

In [101]:
def spam_the_api(start_page, stop_page):
    existing_files = [name.split('_')[1].split('.')[0] for name in os.listdir('axe_specs/') if not name.startswith('.')] # Ignore .DS_Store
    j = 0
    k = 0
    '''Spams the eBay API for pages of AXE DATA'''
    for i in range(start_page+1, stop_page+1):
        page = find_completed(i)
        persist_page_to_json(page)
        for axe in page:
            k += 1
            if axe['itemId'][0] not in existing_files:
                j += 1
                print('Get')
                persist_spec_to_json(get_specs(axe['itemId'][0]))
            else:
                print('Skip')
    print(f'\nChecked {k} axes')
    print(f'\nGot {j} new axes')

Again, you only get 5k API calls per day.

In [106]:
spam_the_api(25, 35)

Skip
Skip
Get
Get
Get
Skip
Skip
Skip
Skip
Skip
Skip
Get
Skip
Skip
Get
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Get
Skip
Skip
Skip
Get
Skip
Skip
Skip
Skip
Skip
Skip
Get
Skip
Skip
Skip
Skip
Get
Get
Skip
Get
Skip
Get
Skip
Skip
Skip
Skip
Skip
Skip
Get
Skip
Get
Get
Skip
Skip
Get
Get
Skip
Skip
Skip
Get
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Get
Skip
Skip
Skip
Skip
Get
Skip
Skip
Skip
Get
Skip
Skip
Get
Skip
Skip
Skip
Skip
Get
Get
Skip
Skip
Skip
Get
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Get
Get
Skip
Get
Skip
Skip
Get
Get
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Get
Get
Skip
Get
Skip
Skip
Skip
Skip
Skip
Skip
Get
Get
Get
Get
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Get
Skip
Get
Skip
Skip
Skip
Get
Skip
Get
Skip
Skip
Skip
Skip
Skip
Skip
Get
Get
Skip
Skip
Skip
Skip
Skip
Skip
Get
Skip
Skip
Skip
Get
Skip
Skip
Get
Skip
Skip
Skip
Get
Skip
Skip
Get
Skip
Skip
Skip
Get
Skip
Skip
Skip
Get
Skip
Skip
Skip
Get
Skip
Get
Skip
Skip
Ge